In [8]:
import random
import matplotlib.pyplot as plt
import time
import numpy as np

In [2]:

class TabuSearch:
    def __init__(self, max_iterations, min_tabu=20, extra_tabu=4):
        self.max_iterations = max_iterations
        self.min_tabu = min_tabu
        self.extra_tabu = extra_tabu

    def solve(self, problem):
        rand = random.Random()
        tabu_list = [0] * len(problem.solution_array)

        solution = problem.solution_array[:]
        fitness = problem.calculate_fitness()

        for k in range(1, self.max_iterations + 1):
            best_move = self.find_best_move(problem, k, tabu_list, fitness)
            problem.solution_array = best_move['best_solution_in_iteration']
            problem.calculate_fitness()
            tabu_list[best_move['best_index_in_iteration'] - 1] = k + self.min_tabu + rand.randint(0, self.extra_tabu)

            if best_move['best_fitness_in_iteration'] < fitness:
                solution = best_move['best_solution_in_iteration']
                fitness = best_move['best_fitness_in_iteration']

        problem.solution_array = solution

        return {'solution': solution, 'fitness': fitness}

    def find_best_move(self, problem, iteration, tabu_list, fitness):
        best_fitness_in_iteration = float('inf')
        best_solution_in_iteration = problem.solution_array[:]
        best_index_in_iteration = 1

        for i in range(1, len(best_solution_in_iteration) + 1):
            possible_solution = self.flip(problem, i)
            possible_fitness = self.value_flip(problem, i)

            if iteration < tabu_list[i - 1] and not possible_fitness < fitness:
                continue

            if possible_fitness < best_fitness_in_iteration:
                best_fitness_in_iteration = possible_fitness
                best_solution_in_iteration = possible_solution
                best_index_in_iteration = i

        return {
            'best_fitness_in_iteration': best_fitness_in_iteration,
            'best_solution_in_iteration': best_solution_in_iteration,
            'best_index_in_iteration': best_index_in_iteration
        }

    def flip(self, problem, index):
        flipped_solution = problem.solution_array[:]
        flipped_solution[index - 1] *= -1
        return flipped_solution

    def value_flip(self, problem, value_index):
        fitness = 0.0
        for p in range(len(problem.solution_array) - 1):
            value = problem.autocorrelations[p]
            if p < len(problem.solution_array) - value_index:
                value -= 2 * problem.autocorrelation_products[p][value_index - 1]

            if p < value_index - 1:
                value -= 2 * problem.autocorrelation_products[p][value_index - 1 - p - 1]

            fitness += value ** 2
        return fitness


In [3]:
class LADS:
    def __init__(self, solution_length=10):
        self.solution_length = solution_length
        self.solution_array = []
        self.autocorrelation_products = []  # T(s)
        self.autocorrelations = []          # C(s)

    def calculate_fitness(self):
        self.calculate_aperiodic_autocorrelation()
        return sum(c * c for c in self.autocorrelations)

    def generate_random_solution(self):
        self.solution_array = [random.choice([-1, 1]) for _ in range(self.solution_length)]

    def calculate_aperiodic_autocorrelation(self):
        self.autocorrelation_products = []
        self.autocorrelations = []
        for k in range(1, self.solution_length):
            products = [self.solution_array[i] * self.solution_array[i + k] for i in range(self.solution_length - k)]
            self.autocorrelation_products.append(products)
            self.autocorrelations.append(sum(products))


In [6]:
class SearchRunner:
    def run(self, iterations, problem, solver, search_iterations, min_tabu, extra_tabu, time_limit_in_seconds):
        results = []
        solver.max_iterations = search_iterations
        solver.min_tabu = min_tabu
        solver.extra_tabu = extra_tabu

        for i in range(iterations):
            start_time = time.time()
            print(f"Algorithm run number {i}")
            best_result = None
            number_of_tries = 0

            while time.time() - start_time < time_limit_in_seconds:
                number_of_tries += 1
                result = solver.solve(LADS(problem))  # Assuming LADS is a class that implements IProblem
                if best_result is None or result > best_result:
                    best_result = result

            print(f"Number of tries {number_of_tries} for max iterations {search_iterations}")
            results.append(best_result)

        return results

    def run_with_multiple_iterations(self, iterations, problem, solver, search_iterations_list, min_tabu, extra_tabu, time_limit_in_seconds):
        output = []
        for search_iterations in search_iterations_list:
            results = self.run(iterations, LADS(problem), solver, search_iterations, min_tabu, extra_tabu, time_limit_in_seconds)
            output.append(results)
        return output

    def run_with_multiple_min_tabu(self, iterations, problem, solver, search_iteration, min_tabu_list, extra_tabu, time_limit_in_seconds):
        output = []
        for min_tabu in min_tabu_list:
            results = self.run(iterations, LADS(problem), solver, search_iteration, min_tabu, extra_tabu, time_limit_in_seconds)
            output.append(results)
        return output

    def run_with_multiple_extra_tabu(self, iterations, problem, solver, search_iteration, min_tabu, extra_tabu_list, time_limit_in_seconds):
        output = []
        for extra_tabu in extra_tabu_list:
            results = self.run(iterations, LADS(problem), solver, search_iteration, min_tabu, extra_tabu, time_limit_in_seconds)
            output.append(results)
        return output


In [9]:
class ResultModel:
    def __init__(self, fitness):
        self.fitness = fitness

class PlotGenerator:
    def generate_single_box_and_whiskers(self, results, plot_name="results"):
        ys = [r.fitness for r in results]

        plt.title("Solution Energy")
        plt.ylabel("Solution Energy")

        plt.boxplot(ys, labels=['Scores'])

        plt.savefig(f"{plot_name}.png")

    def generate_multiple_box_and_whiskers(self, all_results, x_values, plot_title, plot_name="results"):
        data = []
        for results in all_results:
            ys = [r.fitness for r in results]
            data.append(ys)

        plt.title(plot_title)
        plt.ylabel("Solution Energy")

        plt.boxplot(data, labels=[str(x) for x in x_values])

        plt.legend()
        plt.grid(linestyle=':', axis='y')

        plt.savefig(f"{plot_name}.png")

# Example usage
# results = [ResultModel(fitness) for fitness in some_fitness_values]
# plot_generator = PlotGenerator()
# plot_generator.generate_single_box_and_whiskers(results)
# plot_generator.generate_multiple_box_and_whiskers(all_results, x_values, "Some Plot Title")


In [10]:

class PlotGenerator:
    # Implement the GenerateSingleBoxAndWhiskers and GenerateMultipleBoxAndWhiskers methods
    pass

def main():
    problem = LADS(100)
    problem.generate_random_solution()

    search = TabuSearch(200, 20, 4)

    runner = SearchRunner()
    plot_generator = PlotGenerator()

    # Uncomment and implement these lines according to your actual algorithm and plotting logic
    # results = runner.run(100, problem, search, 200)
    # plot_generator.generate_single_box_and_whiskers(results, "results")

    algorithm_runs = 50
    time_limit_in_seconds = 120
    search_iterations = [900, 1100, 1300, 1500]
    multiple_results_iterations = runner.run_with_multiple_iterations(
        algorithm_runs, problem, search, search_iterations, 20, 4, time_limit_in_seconds)
    plot_generator.generate_multiple_box_and_whiskers(multiple_results_iterations, search_iterations, 
                                                      "Solution energy for different values of search iterations", 
                                                      "multiple results iterations")

    # Implement similar logic for minTabus and extraTabus

if __name__ == "__main__":
    main()


Algorithm run number 0


TypeError: 'LADS' object cannot be interpreted as an integer